In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf
import time

2024-03-08 18:18:54.012984: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.11.0'

In [3]:
from tensorflow.keras.utils import to_categorical as labelEncoding 

T = 15 # terminus_length

X1 = np.load('bpf-740.npy')


X1 = X1[:376,:T,:]


Y  = [1 for _ in range(376)]

Y = labelEncoding(Y, dtype=int)

print(X1.shape)
print(Y.shape)

(376, 15, 20)
(376, 2)


In [4]:
print(X1.shape)
print(X1[0].shape)
print(X1[0])

BUFFER_SIZE = 1000
BATCH_SIZE = 32

train_dataset = tf.data.Dataset.from_tensor_slices(X1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

(376, 15, 20)
(15, 20)
[[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]


2024-03-08 18:19:02.135014: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-08 18:19:02.138701: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
# Deep Neural Networks:
import tensorflow as tf; print('We\'re using TF-{}.'.format(tf.__version__))
# import keras; print('We\'re using Keras-{}.'.format(keras.__version__))
from tensorflow.keras.layers import (Input, Dense, Dropout, Flatten, BatchNormalization,
                                     Conv1D, Conv2D, MaxPooling1D, MaxPooling2D,
                                     LSTM, GRU, Embedding, Bidirectional, Concatenate)
from tensorflow.keras.regularizers import (l1, l2, l1_l2)
from tensorflow.keras.optimizers import (RMSprop, Adam, SGD)
from tensorflow.keras.models import (Sequential, Model)

# Core:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interp
import matplotlib.patches as patches

# Performance:
from sklearn.metrics import (confusion_matrix, classification_report, matthews_corrcoef, precision_score, roc_curve, auc)
from sklearn.model_selection import (StratifiedKFold, KFold, train_test_split)

#Utilities:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical as labelEncoding   # Usages: Y = labelEncoding(Y, dtype=int)
from tensorflow.keras.utils import plot_model   

We're using TF-2.11.0.


In [6]:
def build_discriminator():
    ### Head-1:
    input1 = Input(shape=X1[0].shape)

    x = Conv1D(filters=10, kernel_size=4, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(input1)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.80)(x)

    x = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.70)(x)

    head1 = Flatten()(x)

    # merge
    merge = Concatenate()([head1])

    output = Dense(units=8, activation='relu', kernel_regularizer=l2(l=0.01))(merge)
    output = BatchNormalization()(output)
    output = Dropout(rate=0.70)(output)

    output = Dense(units=2, activation='softmax')(output)

    return Model(inputs=[input1], outputs=[output])

In [7]:
from Bio.Seq import Seq
from Bio.SeqUtils import ProtParam
from Bio.Align import substitution_matrices
from Bio.SubsMat import MatrixInfo


# Function to convert binary profile feature to amino acid sequence
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        index = row.argmax()
        sequence += amino_acids[index]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row)
    return blosum_matrix

/home/homayoun/anaconda3/envs/acp-proj/lib/python3.10/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [51]:
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        i = 0
        for item in row:
            i += 1
            if item.numpy() == 1:
                sequence += amino_acids[i]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row[:20])
    return blosum_matrix

def bpf_to_blosum_layer(seqs_bpf):
    seqs_blosum = []
    for bpf in seqs_bpf:
        str_seq = bpf_to_sequence(bpf)
        seqs_blosum.append(seq_to_blosum(str_seq))

    print(seqs_blosum)
    return seqs_blosum

def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(512))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(25 * 20, activation='sigmoid'))
    model.add(layers.Reshape((25, 20)))
    return model
    

    # model = Sequential([
    #     layers.Input(shape=(latent_dim,)),
    #     layers.Dense(128, activation='relu'),
    #     layers.Dense(256, activation='relu'),
    #     layers.Dense(output_shape[0] * output_shape[1], activation='sigmoid'),
    #     layers.Reshape(output_shape),
    #     layers.Lambda(lambda x: tf.one_hot(tf.argmax(x, axis=-1), depth=output_shape[1])),
    #     layers.Lambda(lambda x: tf.cast(x, dtype=tf.int32)),
    # ])

    # return model

In [52]:
# Define the dimensions
latent_dim = 100  # Dimensionality of the latent space
batch_size = 1
output_shape = (25, 20)  # Desired output shape of the matrix

generator = build_generator()

noise = tf.random.normal([batch_size, latent_dim])

generated_matrix = generator(noise, training=False)

generated_matrix

<tf.Tensor: shape=(1, 25, 20), dtype=float32, numpy=
array([[[0.59916043, 0.53571814, 0.40263578, 0.61033857, 0.5403845 ,
         0.5831024 , 0.48558527, 0.4782551 , 0.5556825 , 0.48901847,
         0.5192514 , 0.64240897, 0.4109416 , 0.47700214, 0.5168907 ,
         0.49550176, 0.64228076, 0.44921458, 0.6389669 , 0.5415116 ],
        [0.42288995, 0.5490418 , 0.59424645, 0.47959942, 0.57634515,
         0.6427886 , 0.4730355 , 0.41809404, 0.5134199 , 0.50252575,
         0.42812222, 0.44326895, 0.5549914 , 0.5256387 , 0.58247584,
         0.48176795, 0.47893462, 0.51944   , 0.56293356, 0.3404871 ],
        [0.41126373, 0.4184705 , 0.5213183 , 0.57456857, 0.59182966,
         0.55503994, 0.5803567 , 0.4063949 , 0.5829616 , 0.4649232 ,
         0.5246829 , 0.45952612, 0.57522774, 0.49405873, 0.52228445,
         0.5045324 , 0.4035373 , 0.5319075 , 0.55955607, 0.55667794],
        [0.49701828, 0.52970284, 0.49671882, 0.46807227, 0.5598332 ,
         0.5442624 , 0.4863615 , 0.6514802 , 0.

In [54]:
discriminator = build_discriminator()
discriminator.load_weights('./acp_mhcnn_weights.h5')

In [55]:
prediction = discriminator.predict([X1[:5,:,:]])
prediction

1/1 [==============================] - 0s 111ms/step


array([[0.30780783, 0.6921922 ],
       [0.17504199, 0.824958  ],
       [0.31832418, 0.68167585],
       [0.18360284, 0.81639713],
       [0.19149974, 0.80850023]], dtype=float32)

In [57]:
noise = tf.random.normal([5, 100])
fake_seqs = generator(noise, training=False)

prediction = discriminator.predict([fake_seqs[:,:15,:]])
print(prediction)

1/1 [==============================] - 0s 28ms/step
[[9.9999988e-01 8.5324515e-08]
 [9.9999964e-01 3.3498407e-07]
 [9.9999928e-01 6.6238709e-07]
 [9.9999940e-01 6.4296410e-07]
 [9.9999964e-01 3.1736076e-07]]


In [58]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [59]:
def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy()(tf.ones_like(fake_output), fake_output)

# def generator_loss(disc_output):
#     batch_size = tf.shape(disc_output)[0]
#     num_classes = tf.shape(disc_output)[1]
#     desired_output = tf.concat([tf.zeros((batch_size, 1)), tf.ones((batch_size, num_classes - 1))], axis=1)
    
#     # Define binary cross entropy loss
#     bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    
#     # Calculate loss
#     loss = bce(desired_output, disc_output)
    
#     return loss

In [60]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [61]:
noise = tf.random.normal([5, 100])
fake_seqs = generator(noise, training=False)

fake_prediction = discriminator.predict([fake_seqs[:,:15,:]])
real_prediction = discriminator.predict([X1[:5]])
print(fake_prediction)
print(real_prediction)

# disc_loss = discriminator_loss(Y[:5], real_prediction)
gen_loss = generator_loss(fake_prediction)
print(gen_loss)
disc_loss = discriminator_loss(real_prediction, fake_prediction)
print(disc_loss)

1/1 [==============================] - 0s 51ms/step
[[9.9999988e-01 1.1416962e-07]
 [9.9999952e-01 4.5712932e-07]
 [9.9999952e-01 4.4334897e-07]
 [9.9999928e-01 6.6139603e-07]
 [9.9999893e-01 1.0680716e-06]]
[[0.30780783 0.6921922 ]
 [0.17504199 0.824958  ]
 [0.31832418 0.68167585]
 [0.18360284 0.81639713]
 [0.19149974 0.80850023]]
tf.Tensor(7.1930757, shape=(), dtype=float32)
tf.Tensor(8.051295, shape=(), dtype=float32)


In [62]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [63]:
def generate_and_save_images(model, epoch, test_input):
    pass

In [64]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32

EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [67]:
@tf.function
def train_step(peptides):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_bpf = generator(noise, training=True)

      real_output = discriminator(peptides, training=True)
      fake_output = discriminator(generated_bpf[:,:15,:], training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for seq_batch in dataset:
      train_step(seq_batch)

    # Produce images for the GIF as you go
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  generate_and_save_images(generator,
                           epochs,
                           seed)

In [68]:
train(train_dataset, EPOCHS)

Time for epoch 1 is 3.6060619354248047 sec
Time for epoch 2 is 0.640122652053833 sec
Time for epoch 3 is 0.607396125793457 sec
Time for epoch 4 is 0.3804740905761719 sec
Time for epoch 5 is 0.3364682197570801 sec
Time for epoch 6 is 0.46462416648864746 sec
Time for epoch 7 is 0.552495002746582 sec
Time for epoch 8 is 0.8124563694000244 sec
Time for epoch 9 is 0.3494689464569092 sec
Time for epoch 10 is 0.5501284599304199 sec
Time for epoch 11 is 0.5289163589477539 sec
Time for epoch 12 is 0.5576207637786865 sec
Time for epoch 13 is 0.4158895015716553 sec
Time for epoch 14 is 0.8341262340545654 sec
Time for epoch 15 is 1.0258681774139404 sec
Time for epoch 16 is 0.9606630802154541 sec
Time for epoch 17 is 1.093747615814209 sec
Time for epoch 18 is 1.3936424255371094 sec
Time for epoch 19 is 1.2460801601409912 sec
Time for epoch 20 is 0.5324950218200684 sec
Time for epoch 21 is 0.5579359531402588 sec
Time for epoch 22 is 0.6051826477050781 sec
Time for epoch 23 is 0.7200794219970703 sec


In [45]:
noise = tf.random.normal([5, 100])
#, tf.GradientTape() as disc_tape
peptides = X1[:5, :15, :]
with tf.GradientTape() as gen_tape:
    generated_bpf = generator(noise, training=True)

    # real_output = discriminator(peptides, training=True)
    fake_output = discriminator(generated_bpf[:,:15,:], training=True)

    gen_loss = generator_loss(fake_output)
    # disc_loss = discriminator_loss(real_output, fake_output)
    
    print(gen_loss)
    # print(disc_loss)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    # gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    print(gradients_of_generator)

tf.Tensor(0.9108116, shape=(), dtype=float32)
[None, None, None, None, None, None, None, None, None]


In [70]:
noise = tf.random.normal([100, 100])

generated_bpf = generator(noise, training=True)

fake_output = discriminator(generated_bpf[:,:15,:], training=True)

for key, output in enumerate(fake_output):
    if output[1] > 0.8:
        print(generated_bpf[key])
        break

tf.Tensor(
[[0.68451047 0.5047946  0.74413264 0.67703605 0.6957978  0.4955305
  0.3093026  0.4925447  0.31566343 0.5124231  0.4425264  0.5000091
  0.42750183 0.653235   0.44780466 0.63958216 0.6513634  0.32214785
  0.591771   0.46277422]
 [0.19686647 0.5235408  0.6679461  0.37176424 0.5139134  0.38612974
  0.5833974  0.2920839  0.36459178 0.5819297  0.4758363  0.49902785
  0.22876447 0.47125256 0.6393183  0.21300545 0.58081186 0.28430164
  0.62339056 0.43651858]
 [0.69651693 0.6485016  0.5314197  0.5258587  0.44587508 0.67231584
  0.6153117  0.489702   0.64680994 0.40624878 0.3103658  0.4542466
  0.4417386  0.36703476 0.27699602 0.66781735 0.6951212  0.44777852
  0.8215833  0.7288543 ]
 [0.49689484 0.6939338  0.4545324  0.7042392  0.64587253 0.3566483
  0.39660743 0.6670516  0.38430947 0.4760153  0.34967077 0.60703844
  0.38835892 0.37492418 0.59543157 0.21528035 0.5436811  0.52745515
  0.6851465  0.5640824 ]
 [0.58024496 0.401263   0.17889473 0.40008113 0.5205328  0.58030105
  0.64176